In [2]:
!pip install python-telegram-bot nest_asyncio yfinance --quiet

In [3]:
!pip install pandas numpy scikit-learn prophet tensorflow --quiet

## Подключение логгирования

In [5]:
import logging
import sys

def setup_logger(logfile='bot.log'):
    # Создаём логгер
    logger = logging.getLogger('stockbot')
    logger.setLevel(logging.DEBUG)  # Можно INFO, WARNING, ERROR

    # Формат логов
    formatter = logging.Formatter('%(asctime)s [%(levelname)s] %(message)s')

    # Хэндлер для файла
    fh = logging.FileHandler(logfile, encoding='utf-8')
    fh.setLevel(logging.DEBUG)
    fh.setFormatter(formatter)
    logger.addHandler(fh)

    # Хэндлер для консоли (stdout, чтобы видеть в ноутбуке)
    ch = logging.StreamHandler(sys.stdout)
    ch.setLevel(logging.INFO)
    ch.setFormatter(formatter)
    logger.addHandler(ch)

    return logger

logger = setup_logger()

## Работа с API Yahoo Finance

In [7]:
import os

tickets_path = os.path.join('./', 'yahoo-cache')

def get_ticket_filepath(ticker):
    return os.path.join(tickets_path, f"{ticker}.csv")

def get_ticket_plot_filepath(ticker):
    return os.path.join(tickets_path, f"{ticker}.png")

In [8]:
import yfinance as yf

def is_valid_ticker(ticker):
    try:
        hist = yf.download(ticker, period="1mo")
        logger.info(f"Проверка тикера {ticker}: найдено строк {len(hist)}")
        return not hist.empty
    except Exception as e:
        return False

In [9]:
import pandas as pd
from datetime import datetime, timedelta

def clean_ticker_csv(filename):
    df = pd.read_csv(filename, skiprows=2)
    df.columns = ['Date', 'close', 'high', 'low', 'open', 'volume']
    df.to_csv(filename, index=False)
    return df

def save_ticker_history(ticker):
    end_date = datetime.today()
     # 2 года
    start_date = end_date - timedelta(days=730)

    ticker_obj = yf.Ticker(ticker)
    hist = yf.download(ticker, start=start_date.strftime('%Y-%m-%d'), end=end_date.strftime('%Y-%m-%d'))

    hist.to_csv(get_ticket_filepath(ticker))
    clean_ticker_csv(get_ticket_filepath(ticker))

In [10]:
def file_exists(ticker):
    return os.path.isfile(get_ticket_filepath(ticker))

## Построение моделей

In [12]:
import pandas as pd
import numpy as np
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
from sklearn.preprocessing import StandardScaler
from prophet import Prophet
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

def forecasting_pipeline(ticker, test_size=30, n_lags=5, n_steps=10, epochs=20):
    # 1. Загрузка данных
    filename = get_ticket_filepath(ticker)
    df = pd.read_csv(filename, parse_dates=['Date'])
    df = df.sort_values('Date')
    df = df.dropna()
    series = df['close'].values
    dates = df['Date'].values

    # 2. Train/test split
    train, test = series[:-test_size], series[-test_size:]
    train_dates, test_dates = dates[:-test_size], dates[-test_size:]

    # 3. Ridge Regression с лагами
    def create_lag_features(series, n_lags=5):
        X, y = [], []
        for i in range(n_lags, len(series)):
            X.append(series[i-n_lags:i])
            y.append(series[i])
        return np.array(X), np.array(y)

    X_train, y_train = create_lag_features(train, n_lags)
    X_test, y_test = create_lag_features(np.concatenate([train[-n_lags:], test]), n_lags)

    ridge = Ridge()
    ridge.fit(X_train, y_train)
    ridge_pred = ridge.predict(X_test)

    # 4. Prophet
    df_prophet = pd.DataFrame({'ds': df['Date'], 'y': df['close']})
    train_prophet = df_prophet.iloc[:-test_size]
    model_prophet = Prophet()
    model_prophet.fit(train_prophet)
    future = model_prophet.make_future_dataframe(periods=test_size)
    forecast = model_prophet.predict(future)
    prophet_pred = forecast['yhat'].iloc[-test_size:].values

    # 5. LSTM
    scaler = StandardScaler()
    train_scaled = scaler.fit_transform(train.reshape(-1, 1))
    test_scaled = scaler.transform(test.reshape(-1, 1))

    def create_lstm_features(series, n_steps=10):
        X, y = [], []
        for i in range(n_steps, len(series)):
            X.append(series[i-n_steps:i])
            y.append(series[i])
        return np.array(X), np.array(y)

    X_train_lstm, y_train_lstm = create_lstm_features(train_scaled, n_steps)
    X_test_lstm, y_test_lstm = create_lstm_features(np.concatenate([train_scaled[-n_steps:], test_scaled]), n_steps)

    model = Sequential([
        LSTM(32, input_shape=(n_steps, 1)),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mse')
    model.fit(X_train_lstm, y_train_lstm, epochs=epochs, batch_size=16, verbose=0)

    lstm_pred_scaled = model.predict(X_test_lstm).flatten()
    lstm_pred = scaler.inverse_transform(lstm_pred_scaled.reshape(-1, 1)).flatten()

    # 6. Метрики
    ridge_rmse = mean_squared_error(y_test, ridge_pred, squared=False)
    ridge_mape = mean_absolute_percentage_error(y_test, ridge_pred)
    prophet_rmse = mean_squared_error(test, prophet_pred, squared=False)
    prophet_mape = mean_absolute_percentage_error(test, prophet_pred)
    lstm_rmse = mean_squared_error(y_test, lstm_pred, squared=False)
    lstm_mape = mean_absolute_percentage_error(y_test, lstm_pred)

    metrics = pd.DataFrame({
        'model': ['Ridge', 'Prophet', 'LSTM'],
        'RMSE': [ridge_rmse, prophet_rmse, lstm_rmse],
        'MAPE': [ridge_mape, prophet_mape, lstm_mape]
    })
    print("Метрики на тесте:\n", metrics)

    # 7. Добавляем предсказания в DataFrame
    result_len = min(len(y_test), len(ridge_pred), len(prophet_pred[-len(y_test):]), len(lstm_pred), len(test_dates[n_lags:]))
    result = pd.DataFrame({
        'date': test_dates[n_lags:][:result_len],
        'actual': y_test[:result_len],
        'ridge_pred': ridge_pred[:result_len],
        'prophet_pred': prophet_pred[-result_len:],
        'lstm_pred': lstm_pred[:result_len]
    })

    # 8. Выбор лучшей модели (по RMSE)
    best_idx = metrics['RMSE'].idxmin()
    best_model = metrics.loc[best_idx, 'model']
    print(f"Лучшая модель: {best_model}")

    # 9. Переобучение лучшей модели на всём датасете и прогноз на 30 дней вперёд
    if best_model == 'Ridge':
        X_full, y_full = create_lag_features(series, n_lags)
        ridge.fit(X_full, y_full)
        last_vals = series[-n_lags:].tolist()
        ridge_forecast = []
        for _ in range(30):
            x_input = np.array(last_vals[-n_lags:]).reshape(1, -1)
            pred = ridge.predict(x_input)[0]
            ridge_forecast.append(pred)
            last_vals.append(pred)
        forecast_dates = pd.date_range(df['Date'].iloc[-1] + pd.Timedelta(days=1), periods=30)
        forecast_df = pd.DataFrame({'date': forecast_dates, 'best_pred': ridge_forecast})

    elif best_model == 'Prophet':
        model_prophet = Prophet()
        model_prophet.fit(df_prophet)
        future = model_prophet.make_future_dataframe(periods=30)
        forecast = model_prophet.predict(future)
        forecast_dates = forecast['ds'].iloc[-30:].values
        forecast_df = pd.DataFrame({'date': forecast_dates, 'best_pred': forecast['yhat'].iloc[-30:].values})

    elif best_model == 'LSTM':
        scaler_full = StandardScaler()
        series_scaled = scaler_full.fit_transform(series.reshape(-1, 1))
        X_full_lstm, y_full_lstm = create_lstm_features(series_scaled, n_steps)
        model = Sequential([
            LSTM(32, input_shape=(n_steps, 1)),
            Dense(1)
        ])
        model.compile(optimizer='adam', loss='mse')
        model.fit(X_full_lstm, y_full_lstm, epochs=epochs, batch_size=16, verbose=0)
        last_vals = series_scaled[-n_steps:].tolist()
        lstm_forecast = []
        for _ in range(30):
            x_input = np.array(last_vals[-n_steps:]).reshape(1, n_steps, 1)
            pred_scaled = model.predict(x_input, verbose=0)[0, 0]
            pred = scaler_full.inverse_transform([[pred_scaled]])[0, 0]
            lstm_forecast.append(pred)
            last_vals.append([pred_scaled])
        forecast_dates = pd.date_range(df['Date'].iloc[-1] + pd.Timedelta(days=1), periods=30)
        forecast_df = pd.DataFrame({'date': forecast_dates, 'best_pred': lstm_forecast})

    else:
        raise ValueError("Unknown model")

    return result, metrics, best_model, forecast_df, df

In [13]:
import matplotlib.pyplot as plt

def plot_forecast(df, forecast_df, ticker):
    plt.figure(figsize=(12, 6))
    plt.plot(df['Date'], df['close'], label='Исторические данные', color='blue')
    plt.plot(forecast_df['date'], forecast_df['best_pred'], label='Прогноз на 30 дней', color='red')
    plt.xlabel('Дата')
    plt.ylabel('Цена акции')
    plt.title(f'Прогноз цены акций {ticker.upper()} на 30 дней')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    filename = get_ticket_plot_filepath(ticker)
    plt.savefig(filename)
    plt.close()


In [14]:
# Пример использования:
# result, metrics, best_model, forecast_df, df = forecasting_pipeline('AAPL')
# print(result.tail())
# print(metrics)
# print(forecast_df)
# plot_forecast(df, forecast_df, 'AAPL')

In [15]:
from scipy.signal import find_peaks
import numpy as np

def find_trade_points(prices):
    peaks, _ = find_peaks(prices, prominence=0.05)
    valleys, _ = find_peaks(-prices, prominence=0.05)
    return valleys, peaks

In [16]:
def simulate_trading(prices, amount, dates=None):
    valleys, peaks = find_trade_points(prices)
    # Сортируем экстремумы по времени
    trade_points = sorted(list(valleys) + list(peaks))
    trade_points.sort()
    profit = 0
    actions = []
    i = 0
    while i < len(trade_points) - 1:
        buy_idx = trade_points[i]
        sell_idx = trade_points[i+1]
        if buy_idx in valleys and sell_idx in peaks and sell_idx > buy_idx:
            buy_price = prices[buy_idx]
            sell_price = prices[sell_idx]
            shares = amount / buy_price
            profit += (sell_price - buy_price) * shares
            actions.append({
                'buy_date': dates[buy_idx] if dates is not None else buy_idx,
                'buy_price': buy_price,
                'sell_date': dates[sell_idx] if dates is not None else sell_idx,
                'sell_price': sell_price,
                'profit': (sell_price - buy_price) * shares
            })
            i += 2  # переходим к следующей паре
        else:
            i += 1  # если не пара "минимум-максимум", двигаемся дальше

    if len(actions) == 0 and len(prices) > 1:
        buy_idx = 0
        sell_idx = len(prices) - 1
        buy_price = prices[buy_idx]
        sell_price = prices[sell_idx]
        shares = amount / buy_price
        profit = (sell_price - buy_price) * shares
        actions.append({
            'buy_date': dates[buy_idx] if dates is not None else buy_idx,
            'buy_price': buy_price,
            'sell_date': dates[sell_idx] if dates is not None else sell_idx,
            'sell_price': sell_price,
            'profit': profit
        })
    
    return profit, actions

## Telegram bot

In [18]:
from telegram import Update
from telegram.ext import Application, CommandHandler, MessageHandler, ContextTypes, filters

In [19]:
TELEGRAM_TOKEN = '8482085191:AAH5khDSXWXwDB02Gp5157bSennnXkT3Vog'

In [20]:
async def start(update: Update, context: ContextTypes.DEFAULT_TYPE):
    await update.message.reply_text(
        "Привет! Я бот для анализа и прогнозирования американских акций.\n"
        "Введите тикер компании (например, AAPL) и сумму для инвестиций в долларах через пробел.\n"
        "Пример 1: AAPL 1000\n"
        "Пример 2: NVDA 325"
        "Бот не является финансовым инструментом, результаты носят исключительно учебный характер."
    )

async def handle_message(update: Update, context: ContextTypes.DEFAULT_TYPE):
    try:
        text = update.message.text.strip()
        ticker, amount = text.split(' ')
        ticker = ticker.upper()
        # семантическая валидация ввода    
        try:
            amount = int(amount)
        except ValueError as e:
            await update.message.reply_text(f"Проверьте формат: не получилось перевести в число '{amount}'")
            return 
        # логическая валидация ввода + сохранение данных
        if is_valid_ticker(ticker):
            if not file_exists(ticker):
                save_ticker_history(ticker)
                print(f"Данные для {ticker} сохранены.")
            else:
                print(f"Файл для {ticker} уже существует.")
        else:
            await update.message.reply_text(f"Тикет '{ticker}' не найден")
            return 
        
        await update.message.reply_text(f"Тикет: {ticker}\nСумма денег: {amount}$")
        wait_msg = await update.message.reply_text("Проводится расчет стратегии...")
        result, metrics, best_model, forecast_df, df = forecasting_pipeline(ticker)
 
        last_actual = df['close'].iloc[-1]
        last_pred = forecast_df['best_pred'].iloc[-1]
        abs_change = last_pred - last_actual
        rel_change = (abs_change / last_actual) * 100
        if abs_change > 0:
            trend = "вырастут"
        else:
            trend = "упадут"
        msg = (f"Ожидается, что через 30 дней акции {ticker} {trend} на "
               f"{abs(abs_change):.2f}$ ({abs(rel_change):.2f}%) относительно текущей цены.\n"
               f"Текущая цена: {last_actual:.2f}\n"
               f"Прогнозируемая цена: {last_pred:.2f}")
        await update.message.reply_text(msg)
        await wait_msg.delete()
        
        plot_forecast(df, forecast_df, ticker)
        with open(get_ticket_plot_filepath(ticker), 'rb') as photo:
            await update.message.reply_photo(photo, caption=f"Прогноз цены {ticker.upper()} на 30 дней")

        prices = forecast_df['best_pred'].values
        dates = forecast_df['date'].values
        profit, actions = simulate_trading(prices, amount, dates)
        
        strategy_msg = "Рекомендации по торговле:\n"
        if len(actions) == 0:
            strategy_msg += "В прогнозе не найдено подходящих точек для покупки и продажи.\n"
        else:
            a = actions[0]
            if len(actions) == 1 and a['buy_date'] == dates[0] and a['sell_date'] == dates[-1]:
                if a['profit'] > 0:
                    strategy_msg += (
                        "Рынок показывает устойчивый рост. "
                        "Оптимальная стратегия — купить в первый день прогноза и продать в последний.\n"
                    )
                    strategy_msg += (
                        f"Покупка: {pd.to_datetime(a['buy_date']).strftime('%Y-%m-%d')} по {a['buy_price']:.2f}, "
                        f"продажа: {pd.to_datetime(a['sell_date']).strftime('%Y-%m-%d')} по {a['sell_price']:.2f}, "
                        f"прибыль: {a['profit']:.2f}$\n"
                        f"\nОриентировочная суммарная прибыль: {profit:.2f}$"
                    )
                else:
                    strategy_msg += (
                        "Рынок прогнозируется как падающий. "
                        "Покупка акций не рекомендуется."
                    )
            else:
                for a in actions:
                    strategy_msg += (
                        f"Покупка: {pd.to_datetime(a['buy_date']).strftime('%Y-%m-%d')} по {a['buy_price']:.2f}, "
                        f"продажа: {pd.to_datetime(a['sell_date']).strftime('%Y-%m-%d')} по {a['sell_price']:.2f}, "
                        f"прибыль: {a['profit']:.2f}$\n"
                    )
                strategy_msg += f"\nОриентировочная суммарная прибыль: {profit:.2f}$"
        
        await update.message.reply_text(strategy_msg)


    except Exception as e:
        logger.exception(f"Непредвиденная ошибка в handle_message: {e}")
        await update.message.reply_text("Произошла ошибка")
        
   
async def unknown(update: Update, context: ContextTypes.DEFAULT_TYPE):
    await update.message.reply_text("Извините, я не знаю такой команды. Введите тикер и сумму.")

In [43]:
# для предотвращения ошибки
# "This event loop is already running"
import nest_asyncio
nest_asyncio.apply()
import asyncio

from telegram.error import TelegramError

async def error_handler(update, context):
    logger.error(msg="Exception while handling an update:", exc_info=context.error)

async def on_shutdown(application):
    logger.info("Бот остановлен (выключен)")

async def main_async():
    try:
        logger.info("Бот запускается...")
        app = Application.builder().token(TELEGRAM_TOKEN).build()
    
        app.add_handler(CommandHandler("start", start))
        app.add_handler(MessageHandler(filters.TEXT & (~filters.COMMAND), handle_message))
        app.add_handler(MessageHandler(filters.COMMAND, unknown))
    
        app.add_error_handler(error_handler)
    
        logger.info("Бот запущен!")
        await app.run_polling()
    except Exception as e:
        logger.exception(f"Ошибка при запуске main_async: {e}")
    finally:
        logger.info("Бот остановлен (выключен)")     

task = asyncio.create_task(main_async())

2026-01-11 17:38:53,063 [INFO] Бот запускается...
2026-01-11 17:38:53,109 [INFO] Бот запущен!


In [41]:
task.cancel()

False